# Теория
На рисунке изображена схема посещения роботом точек:
![](./Source/Robot_control_algorithm.png)
Движение осуществляется по следующей схеме:
1. Поворот на месте из текущей позы в нужную позу для начала прямолинейного движения к нужной точке
2. Прямолинейное движение к нужной точке
3. Поворот на месте из текущей позы в конечную позу
## Прямолинейное движение
На данный момент я нормально не могу реализовать прямолинейное движение, так что будем делать так: я буду стараться повернуть робота четко в по направляющему вектору $\vec{a} = \begin{pmatrix} target\_x - x  \\ target\_y - y \end{pmatrix}$ без остановки, регулируя силу поворота и силу прямолинейного движения с помощью П-регулятора (пропорционального регулятора).
![](./Source/forward_move.png)
Разобьем воздействие на моторы на 2 компоненты: прямолинейное движение и поворот (с равными весами).
Чтобы использовать П-регулятор, необходимо знать отклонение текущего значения величины от требуемого.
### Прямолинейная компонента:
На оба мотора подается одинаковый сигнал, пропорциональный расстоянию от робота до целевой точки, т.е. $\propto|\vec{a}|$. Обозначим воздействие на левый мотор как $left\_motor$, а на правый, как $right\_motor$, тогда $$left\_motor = k_{forward} \cdot |\vec{a}|$$ $$right\_motor = k_{forward} \cdot |\vec{a}|$$
Т.к. максимальное значение на вход моторов $\in [-1;1]$, необходимо ограничить значение, для этого применим сигмоиду:
$$left\_motor = \sigma(k_{forward} \cdot |\vec{a}|)$$ $$right\_motor = \sigma(k_{forward} \cdot |\vec{a}|)$$
где $\sigma(x)=\frac{1}{1+e^{-x}}$.
### Поворотная компонента:
Для поворота нужно оценить требуемый угол поворота, т.е. получить значение $\delta = \alpha-\beta$, где $\alpha$ – требуемая ориентация, а $\beta$ – текущая ориентация робота.
$\beta$ поступает на вход в метод, а $\alpha$ рассчитывается по следующей формуле:
$$\alpha = \begin{cases}
arctg \left( \frac{a_y}{a_x} \right)\, \text{ если } a_x > 0, \\
\frac{\pi}{2} \cdot sign(a_y), \text{ если } a_x = 0, \\
\pi + arctg \left( \frac{a_y}{a_x} \right), \text{ если } a_x < 0.
\end{cases}$$
![](./Source/alpha.png)
Используем П-регулятор. Т.к. максимальное значение на вход моторов $\in [-1;1]$, необходимо ограничить значение, для этого применим сигмоиду:
$$left\_motor = \sigma \left(k_{rotate} \cdot \delta \right)$$
$$right\_motor = -\sigma \left (k_{rotate} \cdot \delta \right)$$
### Композиция движений:
Сложим эти 2 компоненты с одинаковым весом 1/2, чтобы итоговый сигнал $\in [-1;1]$:
$$left\_motor = \frac{1}{2} \sigma(k_{forward} \cdot |\vec{a}|) + \frac{1}{2} \sigma \left(k_{rotate} \cdot \delta \right)$$
$$right\_motor = \frac{1}{2} \sigma(k_{forward} \cdot |\vec{a}|) - \frac{1}{2} \sigma \left (k_{rotate} \cdot \delta \right)$$
## Поворот
Как вариант - можно просто использовать поворот без прямолинейной составляющей, таким образом, обратная связь будет учитывать только ориентацию, однако на практике вполне вероятна ситуация, когда во время поворота произойдет смещение центра робота из своего первоначального положения, можно запариться и написать алгоритм поворота с учетом обратной связи (оставлю открытое окно для этого ввиде аргументов метода), но пока времени нет, так что забиваем на такой расклад событий.
$$left\_motor = \sigma \left(k_{rotate} \cdot \delta \right)$$
$$right\_motor = -\sigma \left (k_{rotate} \cdot \delta \right)$$
## Конечный алгоритм
1. Ввод цели $(target\_x;target\_y)$ и $target\_orient$
2. Поворот на месте перед началом прямолинейного движения (Осуществляется по циклу, пока не выполнено условие):
    - Ввод текущих координат $(x;y)$ и текущей ориентации $\beta$
    - Рассчет направляющего вектора по формуле $\vec{a} = \begin{pmatrix} target\_x - x  \\ target\_y - y \end{pmatrix}$
    - Вычисление $\alpha$ по формуле $$\alpha = \begin{cases}
        arctg \left( \frac{a_y}{a_x} \right)\, \text{ если } a_x > 0, \\
        \frac{\pi}{2} \cdot sign(a_y), \text{ если } a_x = 0, \\
        \pi + arctg \left( \frac{a_y}{a_x} \right), \text{ если } a_x < 0.
        \end{cases}$$
    - Вычисление $\delta = \alpha - \beta$
    - Проверка условия $\left| \delta \right| \le accuracy\_orient$. Если выполнено, то выход из цикла
    - Иначе рассчет управляющих сигналов на оба мотора:
        $$left\_motor = \sigma \left(k_{rotate} \cdot \delta \right)$$
        $$right\_motor = -\sigma \left (k_{rotate} \cdot \delta \right)$$
3. Прямолинейное движение (Осуществляется по циклу, пока не выполнено условие):
    - Ввод текущих координат $(x;y)$ и текущей ориентации $\beta$
    - Рассчет направляющего вектора по формуле $\vec{a} = \begin{pmatrix} target\_x - x  \\ target\_y - y \end{pmatrix}$
    - Проверка условия $\left| \vec{a} \right| \le accuracy\_pos$. Если выполнено, то выход из цикла
    - Иначе вычисление $\alpha$ по формуле $$\alpha = \begin{cases}
        arctg \left( \frac{a_y}{a_x} \right)\, \text{ если } a_x > 0, \\
        \frac{\pi}{2} \cdot sign(a_y), \text{ если } a_x = 0, \\
        \pi + arctg \left( \frac{a_y}{a_x} \right), \text{ если } a_x < 0.
        \end{cases}$$
    - Вычисление $\delta = \alpha - \beta$
    - Рассчет управляющих сигналов на оба мотора:
        $$left\_motor = \frac{1}{2} \sigma(k_{forward} \cdot |\vec{a}|) + \frac{1}{2} \sigma \left(k_{rotate} \cdot \delta \right)$$
        $$right\_motor = \frac{1}{2} \sigma(k_{forward} \cdot |\vec{a}|) - \frac{1}{2} \sigma \left (k_{rotate} \cdot \delta \right)$$
4. Поворот на месте на целевой угол (Осуществляется по циклу, пока не выполнено условие):
    - Ввод текущих координат $(x;y)$ и текущей ориентации $\beta$
    - Вычисление $\delta = target\_orient - \beta$
    - Проверка условия $\left| \delta \right| \le accuracy\_orient$. Если выполнено, то выход из цикла
    - Иначе рассчет управляющих сигналов на оба мотора:
        $$left\_motor = \sigma \left(k_{rotate} \cdot \delta \right)$$
        $$right\_motor = -\sigma \left (k_{rotate} \cdot \delta \right)$$
5. Отправка ответа об успешном достижении цели

In [17]:
import numpy as np
class RobotMovement:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.phase = 0   #Фаза - это то действие, которое совершает робот
        self.k_forward = 0.01 #Определяются методом тыка
        self.k_rotate = 0.01  #Определяются методом тыка
        
    def sigmoid(x):
        return 1/(1+np.e**(-x))
        
    def set_target(self,target_x,target_y,accuracy_pos,target_orient = False,accuracy_orient = False): #Установить Цель
        self.target_x = target_x
        self.target_y = target_y
        self.target_orient = target_orient
        self.accuracy_pos = accuracy_pos
        self.accuracy_orient = accuracy_orient
        
    def get_target(self): #Получить цель
        return [self.target_x, self.target_y, self.target_orient, self.accuracy_pos, self.accuracy_orient]
        
    def check_target_completion(self): #Проверка, выполнена ли цель
        if self.phase == 3:
            target_completion = True
        else:
            target_completion = False
        return target_completion
        
    def move(self,x,y,orient): #Двигайся к цели!
        if self.phase == 0:   #Фаза начального поворота
            a = np.array([target_x - x, target_y - y]) #Напрявляющий вектор
            #Вычисление угла alpha
            if a[0] > 0:
                alpha = np.atan(a[1]/a[0])
            elif a[0] == 0:
                alpha = np.pi/2 * np.sign(a[1])
            else: #a[0] < 0
                alpha = np.pi + np.atan(a[1]/a[0])

            delta = alpha - orient
            if delta <= self.accuracy_orient: #Проверка на выполнение поворота
                self.phase += 1 #Переход к выполнению следующего действия
                left_motor = 0
                right_motor = 0
            else:
                left_motor = sigmoid(self.k_rotate * delta)
                right_motor = - sigmoid(self.k_rotate * delta)

        
        elif self.phase == 1: #Фаза движения
            a = np.array([target_x - x, target_y - y]) #Напрявляющий вектор
            r = np.sqrt(a[0]**2 + a[1]**2) #Модуль вектора a
            if r <= self.accuracy_pos: #Проверка на выполненение действия
                self.phase += 1 #Переход к выполнению следующего действия
                left_motor = 0
                right_motor = 0
            else:
                #Вычисление угла alpha
                if a[0] > 0:
                    alpha = np.atan(a[1]/a[0])
                elif a[0] == 0:
                    alpha = np.pi/2 * np.sign(a[1])
                else: #a[0] < 0
                    alpha = np.pi + np.atan(a[1]/a[0])

                delta = alpha - orient
                left_motor = 1/2 * sigmoid(self.k_forward * r) + 1/2 * sigmoid(self.k_rotate * delta)
                right_motor = 1/2 * sigmoid(self.k_forward * r) - 1/2 * sigmoid(self.k_rotate * delta)

        
        elif self.phase == 2: #Фаза конечного поворота
            if self.target_orient != False:
                delta = self.target_orient - orient
                if delta <= self.accuracy_orient: #Проверка на выполнение поворота
                    self.phase += 1 #Переход к выполнению следующего действия
                    left_motor = 0
                    right_motor = 0
                else:
                    left_motor = sigmoid(self.k_rotate * delta)
                    right_motor = - sigmoid(self.k_rotate * delta)
            else: #Если конечный поворот не требуется
                self.phase += 1 #Переход к выполнению следующего действия
                left_motor = 0
                right_motor = 0

        
        elif self.phase == 3: #Фаза ожидания следующей цели
            left_motor = 0
            right_motor = 0

        return [left_motor, right_motor]

In [15]:
#Псевдокод управления роботом:
targets = [[1,1,3.4],[10,-66],[74,30,2.3]]
accuracy_pos = 1
accuracy_orient = 0.1
jetbot = RobotMovement()
for t in targets:
    if len(t) == 3:
        jetbot.set_target(t[0],t[1],accuracy_pos,t[2],accuracy_orient)
    elif len(t) == 2:
        jetbot.set_target(t[0],t[1],accuracy_pos)
    while jetbot.check_target_completion() == False:
        x, y, orient = get_pos() #Псевдофункция
        send(jetbot.move(x,y,orient)) #Псевдофункция отправки команд jetbot'у
    

0